COMP-6651 - Algorithm Design Techniques

Project - Clustering Algorithms Analysis



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import AffinityPropagation as SklearnAffinityPropagation

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import os
import kagglehub
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from scipy.spatial.distance import cdist
from sklearn.metrics import (calinski_harabasz_score, silhouette_score,
                             davies_bouldin_score, adjusted_rand_score,
                             normalized_mutual_info_score)


In [ ]:
# Preprocessing for AI Global Index Dataset
def preprocess_ai_index(data):
    # Separate numerical and categorical columns
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    categorical_cols = data.select_dtypes(exclude=[np.number]).columns

    scaler = MinMaxScaler()
    num_scaled = scaler.fit_transform(data[numeric_cols]) if len(numeric_cols) > 0 else np.array([])

    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    cat_encoded = encoder.fit_transform(data[categorical_cols]) if len(categorical_cols) > 0 else np.array([])

    if num_scaled.size and cat_encoded.size:
        return np.hstack((num_scaled, cat_encoded))
    elif num_scaled.size:
        return num_scaled
    else:
        return cat_encoded


In [ ]:
path_iris = kagglehub.dataset_download("himanshunakrani/iris-dataset")
f_path_iris = os.path.join(path_iris, 'iris.csv')
iris_df = pd.read_csv(f_path_iris)
# Extract labels (species)
iris_labels = iris_df['species']
# Remove label from the features
iris_features = iris_df.drop(columns=['species'])
# Convert to numpy
iris_data = iris_features.values

path_ai_index = kagglehub.dataset_download("katerynameleshenko/ai-index")
f_path_ai_index = os.path.join(path_ai_index, 'AI_index_db.csv')
ai_df = pd.read_csv(f_path_ai_index)
ai_df = ai_df.dropna()
# Extract the 'Cluster' column as the label
ai_labels = ai_df['Cluster']
# Remove the 'Cluster' column from the features
ai_df_features = ai_df.drop(columns=['Cluster'])
# Preprocess the remaining features
ai_data = preprocess_ai_index(ai_df_features)
print(ai_df.head())
path_earthquakes= kagglehub.dataset_download("shreyasur965/recent-earthquakes")
f_path_earthquakes = os.path.join(path_earthquakes, 'earthquakes.csv')
earthquake_df = pd.read_csv(f_path_earthquakes)
earthquake_df = earthquake_df[['magnitude', 'felt', 'cdi','mmi','tsunami','sig','depth', 'latitude', 'longitude', 'alert']].dropna()
# Extract the alert labels
earthquake_data_alerts = earthquake_df['alert']
alert_encoded = LabelEncoder().fit_transform(earthquake_data_alerts)
# Remove the label from features
earthquake_data_features = earthquake_df.drop(columns=['alert'])
# Scale the numeric features
earthquake_data = StandardScaler().fit_transform(earthquake_data_features)

100%|██████████| 0.98k/0.98k [00:00<00:00, 430kB/s]

Extracting files...


100%|██████████| 2.38k/2.38k [00:00<00:00, 4.43MB/s]

Extracting files...
                    Country  Talent  Infrastructure  Operating Environment  \
0  United States of America  100.00           94.02                  64.56   
1                     China   16.51          100.00                  91.57   
2            United Kingdom   39.65           71.43                  74.65   
3                    Canada   31.28           77.05                  93.94   
4                    Israel   35.76           67.58                  82.44   

   Research  Development  Government Strategy  Commercial  Total score  \
0    100.00       100.00                77.39      100.00       100.00   
1     71.42        79.97                94.87       44.02        62.92   
2     36.50        25.03                82.82       18.91        40.93   
3     30.67        25.78               100.00       14.88        40.19   
4     32.63        27.96                43.91       27.33        39.89   

         Region                Cluster  Income group   Political r

100%|██████████| 214k/214k [00:00<00:00, 45.0MB/s]

Extracting files...


In [ ]:
import numpy as np

class BASE_AffinityPropagation:
    def __init__(self, damping=0.5, max_iter=200, convergence_iter=15, preference=None):
        self.damping = damping
        self.max_iter = max_iter
        self.convergence_iter = convergence_iter
        self.preference = preference
        self.R = None
        self.A = None
        self.S = None
        self.exemplars_ = None
        self.labels_ = None

    def _compute_similarity(self, X):
        X = np.array(X)
        n_samples = X.shape[0]
        # Compute negative squared Euclidean distance
        S = -np.square(np.linalg.norm(X[:, np.newaxis, :] - X, axis=2))
        # Set preferences (diagonal)
        if self.preference is None:
            preference = np.median(S)
        else:
            preference = self.preference
        np.fill_diagonal(S, preference)
        return S

    def _update_responsibilities(self, S, R, A):
        AS = A + S
        np.fill_diagonal(AS, -np.inf)
        max_AS = np.max(AS, axis=1, keepdims=True)
        new_R = S - max_AS
        # Apply damping
        new_R = self.damping * R + (1 - self.damping) * new_R
        return new_R

    def _update_availabilities(self, R, A):
        n = R.shape[0]
        new_A = np.zeros_like(A)
        # Update a(k,k)
        sum_max_R = np.sum(np.maximum(R, 0), axis=0)
        diag_R = np.diag(R)
        new_A_diag = sum_max_R - np.maximum(0, diag_R)
        np.fill_diagonal(new_A, new_A_diag)

        # Update a(n,k) for n != k
        sum_max_R = np.sum(np.maximum(R, 0), axis=0)
        diag_R = np.diag(R)
        for k in range(n):
            r_kk = diag_R[k]
            sum_other = sum_max_R[k] - np.maximum(R[:, k], 0) - np.maximum(r_kk, 0)
            values = r_kk + sum_other
            new_A[:, k] = np.minimum(0, values)
            new_A[k, k] = new_A_diag[k]  # Ensure diagonal is not overwritten

        # Apply damping
        new_A = self.damping * A + (1 - self.damping) * new_A
        return new_A

    def _get_exemplars(self, R, A):
        diagonal_sum = np.diag(R) + np.diag(A)
        exemplars = np.where(diagonal_sum >= 0)[0].tolist()
        return exemplars

    def fit(self, X):
        X = np.array(X)
        self.S = self._compute_similarity(X)
        n_samples = X.shape[0]
        self.R = np.zeros((n_samples, n_samples))
        self.A = np.zeros((n_samples, n_samples))
        exemplars_history = []
        consecutive_no_changes = 0

        for it in range(self.max_iter):
            new_R = self._update_responsibilities(self.S, self.R, self.A)
            new_A = self._update_availabilities(new_R, self.A)

            current_exemplars = self._get_exemplars(new_R, new_A)
            current_exemplars.sort()
            self.R = new_R
            self.A = new_A

            if exemplars_history:
                previous_exemplars = exemplars_history[-1]
                previous_exemplars.sort()
                if previous_exemplars == current_exemplars:
                    consecutive_no_changes += 1
                else:
                    consecutive_no_changes = 0
                    exemplars_history.append(current_exemplars)
            else:
                exemplars_history.append(current_exemplars)

            if consecutive_no_changes >= self.convergence_iter:
                break

        self.exemplars_ = self._get_exemplars(self.R, self.A)
        self.exemplars_.sort()
        K = self.exemplars_
        n_samples = X.shape[0]

        # Assign labels
        self.labels_ = -np.ones(n_samples, dtype=int)
        exemplar_indices = {k: idx for idx, k in enumerate(K)}
        for n in range(n_samples):
            if n in exemplar_indices:
                self.labels_[n] = exemplar_indices[n]
            else:
                max_sum = -np.inf
                best_k_idx = -1
                for idx, k in enumerate(K):
                    current_sum = self.A[n, k] + self.R[n, k]
                    if current_sum > max_sum:
                        max_sum = current_sum
                        best_k_idx = idx
                self.labels_[n] = best_k_idx

        return self


In [ ]:
def compute_diameter(X, labels):
    """
    Computes the average 'diameter' across all clusters,
    where 'diameter' of a cluster is the maximum distance
    between any two points in that cluster.
    """
    from scipy.spatial.distance import pdist, squareform

    unique_labels = np.unique(labels)
    diameters = []
    for lbl in unique_labels:
        cluster_points = X[labels == lbl]
        if len(cluster_points) > 1:
            # pairwise distances in the cluster
            dist_matrix = squareform(pdist(cluster_points))
            diameters.append(dist_matrix.max())
        else:
            # A single point has diameter 0
            diameters.append(0.0)
    return np.mean(diameters)


def compute_split(X, labels):
    """
    An example 'split' metric: ratio of the size of the largest cluster
    to the size of the smallest cluster. If there's only one cluster, return 1.
    """
    unique_labels, counts = np.unique(labels, return_counts=True)
    if len(counts) < 2:
        return 1.0
    return counts.max() / counts.min()


def evaluate_clustering(X, labels, true_labels=None):
    """
    Compute a set of metrics for the given clustering labels.
    Some metrics require ground truth (true_labels).
    If no true_labels is provided, ARI and MI will be omitted.
    Returns a dict of metric_name -> value.
    """
    metrics_dict = {}

    # Unsupervised metrics
    metrics_dict["Silhouette"] = silhouette_score(X, labels)
    metrics_dict["Davies-Bouldin"] = davies_bouldin_score(X, labels)
    metrics_dict["Calinski-Harabasz"] = calinski_harabasz_score(X, labels)
    metrics_dict["Diameter"] = compute_diameter(X, labels)
    metrics_dict["Split"] = compute_split(X, labels)

    # If we have ground-truth labels, we can compute supervised metrics
    if true_labels is not None:
        metrics_dict["Adjusted Rand Index"] = adjusted_rand_score(true_labels, labels)
        # Using normalized_mutual_info_score as a measure of MI
        metrics_dict["Mutual Information"] = normalized_mutual_info_score(true_labels, labels)

    return metrics_dict


In [ ]:


def run_affinity_experiment(name, X, damping=0.5, max_iter=200, convergence_iter=15,
                            preference=None, true_labels=None):
    print(f"----- {name} -----")

    # Custom Affinity Propagation Implementation
    custom_ap = BASE_AffinityPropagation(
        damping=damping,
        max_iter=max_iter,
        convergence_iter=convergence_iter,
        preference=preference
    )
    custom_ap.fit(X)
    custom_labels = custom_ap.labels_
    n_custom_clusters = len(custom_ap.exemplars_)
    print(f"Custom AP: Number of clusters = {n_custom_clusters}")

    # Scikit-learn's Implementation
    sklearn_ap = SklearnAffinityPropagation(
        damping=damping,
        max_iter=max_iter,
        convergence_iter=convergence_iter,
        preference=preference,
        affinity='euclidean',
        random_state=0
    )
    sklearn_ap.fit(X)
    sklearn_labels = sklearn_ap.labels_
    n_sklearn_clusters = len(sklearn_ap.cluster_centers_)
    print(f"Sklearn AP: Number of clusters = {n_sklearn_clusters}")

    # Evaluation Metrics
    custom_metrics = evaluate_clustering(X, custom_labels, true_labels=true_labels)
    sklearn_metrics = evaluate_clustering(X, sklearn_labels, true_labels=true_labels)

    print("\nCustom AP Metrics:")
    for k, v in custom_metrics.items():
        print(f"  {k}: {v:.4f}")

    print("\nSklearn AP Metrics:")
    for k, v in sklearn_metrics.items():
        print(f"  {k}: {v:.4f}")

    print("-" * 40)
    return custom_labels, sklearn_labels



In [ ]:
print("Running Custom and Sklearn AP on Iris Dataset")
custom_labels_iris, sklearn_labels_iris = run_affinity_experiment(
    "Iris",
    iris_data,
    damping=0.67,
    preference=np.median(-np.square(np.linalg.norm(iris_data[:, np.newaxis] - iris_data, axis=2))) * 0.9,
    true_labels=iris_labels
)

print("\nRunning Custom and Sklearn AP on AI Global Index Dataset")
custom_labels_ai, sklearn_labels_ai = run_affinity_experiment(
    "AI Global Index",
    ai_data,
    damping=0.7,
    preference=np.median(-np.square(np.linalg.norm(ai_data[:, np.newaxis] - ai_data, axis=2)))*1.2,  # Median similarity
    true_labels=ai_labels
)

print("\nRunning Custom and Sklearn AP on Earthquake Dataset")
custom_labels_eq, sklearn_labels_eq = run_affinity_experiment(
    "Earthquake",
    earthquake_data,
    damping=0.9,
    preference=np.median(-np.square(np.linalg.norm(earthquake_data[:, np.newaxis] - earthquake_data, axis=2)))*0.9,  # Controls cluster count
    true_labels=alert_encoded
)

Running Custom and Sklearn AP on Iris Dataset
----- Iris -----
Custom AP: Number of clusters = 2
Sklearn AP: Number of clusters = 7

Custom AP Metrics:
  Silhouette: 0.3212
  Davies-Bouldin: 0.7742
  Calinski-Harabasz: 106.4128
  Diameter: 3.7179
  Split: 3.0541
  Adjusted Rand Index: 0.2485
  Mutual Information: 0.3609

Sklearn AP Metrics:
  Silhouette: 0.4412
  Davies-Bouldin: 0.9984
  Calinski-Harabasz: 392.7419
  Diameter: 1.5167
  Split: 5.5556
  Adjusted Rand Index: 0.5988
  Mutual Information: 0.6900
----------------------------------------

Running Custom and Sklearn AP on AI Global Index Dataset
----- AI Global Index -----
Custom AP: Number of clusters = 5
Sklearn AP: Number of clusters = 9

Custom AP Metrics:
  Silhouette: 0.1152
  Davies-Bouldin: 1.9397
  Calinski-Harabasz: 8.1790
  Diameter: 2.6966
  Split: 4.6000
  Adjusted Rand Index: 0.0287
  Mutual Information: 0.3040

Sklearn AP Metrics:
  Silhouette: 0.2012
  Davies-Bouldin: 1.4626
  Calinski-Harabasz: 8.4031
  Diamet

/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_affinity_propagation.py:140: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


In [ ]:
def select_top_k_features_for_alert(df, alert_col='alert', k=5):
    """
    Filters features based on Mutual Information (MI) with the alert label.
    Steps:
      1) Drop rows with any missing values (to avoid errors).
      2) Label-encode the alert column if it is categorical (e.g., text).
      3) For each feature, compute MI against the alert.
      4) Sort features by MI score, descending.
      5) Return the top k feature names.

    """
    data = df.dropna()

    # Separate the target (alert) from the features
    y = data[alert_col]
    X_df = data.drop(columns=[alert_col])

    # encode alert:
    y_enc = LabelEncoder().fit_transform(y)

    # mutual_info_classif calculates MI between each column and y_enc
    mi_scores = mutual_info_classif(X_df.values, y_enc, discrete_features=False)

    # Pair up (feature_name, mi_score)
    feats_and_scores = list(zip(X_df.columns, mi_scores))

    # Sort by MI descending
    feats_and_scores.sort(key=lambda x: x[1], reverse=True)

    # Pick top k
    top_features = [f[0] for f in feats_and_scores[:k]]

    return top_features





## 4.2 - Select the top 5 features for predicting alert

In [ ]:
top5_features = select_top_k_features_for_alert(
    earthquake_df,  # the original DataFrame with 'alert' and other columns
    alert_col='alert',
    k=5
)
print("Top 5 features most predictive of 'alert':", top5_features)

Top 5 features most predictive of 'alert': ['sig', 'mmi', 'longitude', 'latitude', 'magnitude']


## 4.3 - Re-run the clustering using only top-5 features

In [ ]:
# Subset the DataFrame to those top-5 columns
earthquake_df_top5 = earthquake_df[top5_features]

# 2) Scale the new subset
eq_top5_data = StandardScaler().fit_transform(earthquake_df_top5.values)

print("Running Base/Sklearn Affinity on Earthquake Dataset with Top-5 Features")
_ = run_affinity_experiment(
    "Earthquake (Top-5 Features)",
    eq_top5_data,
    damping=0.7,
    preference=np.median(-np.square(np.linalg.norm(eq_top5_data[:, np.newaxis] - eq_top5_data, axis=2)))* 0.8,  # Controls cluster count
    true_labels=alert_encoded
)


Running Base/Sklearn Affinity on Earthquake Dataset with Top-5 Features
----- Earthquake (Top-5 Features) -----
Custom AP: Number of clusters = 153
Sklearn AP: Number of clusters = 39

Custom AP Metrics:
  Silhouette: -0.1539
  Davies-Bouldin: 1.4893
  Calinski-Harabasz: 20.6830
  Diameter: 0.4492
  Split: 58.0000
  Adjusted Rand Index: -0.0009
  Mutual Information: 0.0797

Sklearn AP Metrics:
  Silhouette: 0.3841
  Davies-Bouldin: 0.8613
  Calinski-Harabasz: 298.9453
  Diameter: 1.5026
  Split: 37.5000
  Adjusted Rand Index: 0.0134
  Mutual Information: 0.1466
----------------------------------------
